<a href="https://colab.research.google.com/github/GaoangLiu/AA_ipynb/blob/master/A_journey_of_predicting_future_sales_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook shows a routine of using machine learning to solve problems. Following topics are covered:
1. data preprocessing
2. data manipulating 
3. building models
4. fine-tuning and improve models.

# Data preprocessing 
When working with machine learning, for most of the time, we're dealing with data cleaning, exploring and merging. Building models and fine-tuning parameters, on the other hand, has been much easier with so many popular libraries, e.g., `Sklearn`, `Tensorflow` and `Pytorch`.

<!-- Data preprocessing is very important -->

In this notebook, we will take the following task [predict future sales](https://www.kaggle.com/c/competitive-data-science-predict-future-sales/notebooks) as an example, to show how to precess data to comply with our needs.

In [0]:
import os 
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import sklearn
import sklearn.metrics 
import sklearn.model_selection
import sklearn.feature_extraction
import sklearn.preprocessing

from absl import logging
import gensim.downloader as api
import matplotlib.pyplot as plt
logging.set_verbosity(logging.INFO)

import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

## File description. 

There are 6 files for this specific task:
1. `sales_train.csv` - the training set. Daily historical data from January 2013 to October 2015.
2. `test.csv` - the test set. You need to forecast the sales for these shops and products for November 2015.
3. `sample_submission.csv` - a sample submission file in the correct format.
4. `items.csv` - supplemental information about the items/products.
5. `item_categories.csv`  - supplemental information about the items categories.
6. `shops.csv` - supplemental information about the shops.

In [0]:
!curl -o sales.zip ali.140714.xyz:8000/sales.zip
!unzip sales.zip
!ls

We craft several methods to explore the data.

In [0]:
def birdview(data):
    print("----------Head 5 Record----------")
    print(data.head(5))
    print("\n-----------Information-----------")
    print(data.info())
    print("\n-----------Data Types-----------")
    print(data.dtypes)
    print("\n----------Missing value-----------")
    print(data.isnull().sum())
    print("\n----------Null value-----------")
    print(data.isna().sum())
    print("\n----------Shape of Data----------")
    print(data.shape)
    
def graph_insight(data):
    print(set(data.dtypes.tolist()))
    df_num = data.select_dtypes(include = ['float64', 'int64'])
    df_num.hist(figsize=(16, 16), bins=50, xlabelsize=8, ylabelsize=8);

def distribution_insight(data, column):
    print("===================================")
    print("Min Value:", data[column].min())
    print("Max Value:", data[column].max())
    print("Average Value:", data[column].mean())
    print("Center Point of Data (median):", data[column].median())
    print("===================================")
    # sns.boxplot(data[column])
    
def remove_duplicate(data):
    print('Before remove shape:', data.shape)
    before = data.shape[0]
    # subset is list where you have to put all column for duplicate check
    # if None, then use all columns 
    data.drop_duplicates(subset=None, keep='first', inplace=True) 
    data.reset_index(drop=True, inplace=True)
    print('After remove shape :', data.shape)
    after = data.shape[0]
    print('Total Duplicate:', before-after)    

## Explore `train.csv`
First, let's find out what we have in this **train** file, is there any missing, duplicate value?. If so, we will dealing with it by: 
* removing duplicates
* removing or replacing outliers, e.g., negative daily sale counts
* 

In [0]:
train = pd.read_csv('data/sales_train.csv')
birdview(train)

A birdview of `train.csv`, we found that:
1. There are outliers, e.g., the 3rd row has a negative `item_cnt_day`
2. There is no missing value or Null value, which is good.

Now, we run `remove_duplicate()` method to remove potential duplicate rows. 

In [0]:
remove_duplicate(train)

We found and removed 6 duplicated rows, not a big deal since we still got 2_935_843 unique rows. 

Next, we need to deal with outliers. A natual way is drawing its distribution graph and see whether this distribution comply with our intuition. 

We've also wrote a method `distribution_insight()` to gain insights on `int | float` datatype columns.

In [0]:
graph_insight(train)
for col in ('date_block_num', 'item_id', 'shop_id', 'item_cnt_day', 'item_price'):
    print(col)
    distribution_insight(train, col)

## Data cleaning 
Dealing with outlier, fill out those empty values, etc. 

There are negative `item_cnt_day` and negative `item_price`, this can not be right. 

Let's deal with negative `item_price` first. 

In [0]:
plt.figure(figsize=(24, 5))
sns.boxplot(train['item_price'])
train[train.item_price <= 0]

There is one row with -1.0 `item_price`, and its `item_id` is 2973. 

By filtering on the samee `item_id` and `date_block_num` (same month): `train[(train.item_id == 2973) & (train.date_block_num == 4)]`, we found that most price during this month is `1249`. So we're going to replace it with `1249`. 

Also, there is an outlier with very large `item_price`: 307980, we simply remove it from dataset

In [0]:
train.loc[train.item_price < 0, 'item_price'] = 1249

train = train[train.item_price < 100_000]
train[train.item_price >= 100_000]

We're done with the `item_price` feature, now we need to fix `item_cnt_day`. 

This problem is a little bit more serious, since we got 7356 rows with negative `item_cnt_day`. Most of them has `-1` as the value. Potential strategies:
1. replace each negative value by its absolution value (maybe caused by writing mistakes)
2. remove them 
3. replace with most frequent `item_cn_day` with same `item_id`

For simplity, we remove those rows.

In [0]:
train[train.item_cnt_day < 0].shop_id.value_counts()
# train[(train.item_cnt_day < 0) & (train.shop_id == 25)]
train[(train.date_block_num == 1) & (train.shop_id == 12)].item_cnt_day.value_counts()
train = train[train.item_cnt_day >= 0]

Some Russian speakers found mislabel on shop ids.
So we correct them in train and test set.

In [0]:
# Якутск Орджоникидзе, 56
train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
train.loc[train.shop_id == 11, 'shop_id'] = 10
test.loc[test.shop_id == 11, 'shop_id'] = 10
# РостовНаДону ТРК "Мегацентр Горизонт" Островной
train.loc[train.shop_id == 40, 'shop_id'] = 39
test.loc[test.shop_id == 40, 'shop_id'] = 39

Now we're done with data cleaning, the next step is tailoring (is this a word) our data so we can build a model and run on it.

# Data manipulating

In [0]:
items = pd.read_csv('data/items.csv')
shops = pd.read_csv('data/shops.csv')
cats  = pd.read_csv('data/item_categories.csv')
test  = pd.read_csv('data/test.csv').set_index('ID')

Observations (by some Russian speakers):

1. Each shop_name starts with the city name.
2. Each category contains type and subtype in its name.

The following codes deal with 
1. `shop_name` (in Russian) from dataset `shops` by extracting the city name from it and label encoding it to numbers
2. `item_category_name` from dataset `cats` by spliting it into `main_type` and `sub_type` and label encoding them
3. dropping `item_name` from dataset `items`

In [0]:
shops.head()

In [0]:
cats.head()

In [0]:
items.head()

Categorize shop names

In [0]:
shops_df = shops 
shops_df.loc[shops.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'

shops_df['city'] = shops['shop_name'].str.split(' ').map(lambda x: x[0])
shops_df['category'] = shops['shop_name'].str.split(' ').map(lambda x: x[1]).astype(str)
shops_df.loc[shops_df.city == '!Якутск', 'city'] = 'Якутск'

top_cats = ['ТЦ', 'ТРК', 'ТРЦ', 'ТК']
shops_df['category'] = shops_df.category.apply(lambda x: x if x in top_cats else 'etc')

shops_df['shop_city'] = sklearn.preprocessing.LabelEncoder().fit_transform(shops_df['city'])
shops_df['shop_category'] = sklearn.preprocessing.LabelEncoder().fit_transform(shops_df['category'])
shops_df = shops_df[['shop_id','shop_city', 'shop_category']]

shops_df.head()

Categorize cats info

In [0]:
cats_df = cats 
cats_df['type_code'] = cats_df.item_category_name.apply(lambda x: x.split(' ')[0]).astype(str)
# Fuse some item_category_name 
cats_df.loc[(cats_df.type_code == 'Игровые') | (cats_df.type_code == 'Аксессуары'), 'type_code'] = 'Игры'
cats_df.loc[cats_df.type_code == 'PC', 'type_code'] = 'Музыка'

top_type_codes = set(['Игры', 'Карты', 'Кино', 'Книги','Музыка', 'Подарки', 'Программы', 'Служебные', 'Чистые'])
cats_df['type_code'] = cats_df.type_code.apply(lambda t: t if t in top_type_codes else 'etc')
cats_df['type_code'] = sklearn.preprocessing.LabelEncoder().fit_transform(cats_df['type_code'])

## Labelencoder item_category_name (sub category) to subtype_code
cats_df['split'] = cats_df.item_category_name.apply(lambda x: x.split('-'))
cats_df['subtype'] = cats_df['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
cats_df['subtype_code'] = sklearn.preprocessing.LabelEncoder().fit_transform(cats_df['subtype'])

cats_df = cats_df[['item_category_id','type_code', 'subtype_code']]
cats_df.head()
# cats_df.type_code.value_counts()

Categorize item info

In [167]:
import collections
import operator 

items_df = items
items_df['item_name'] = items_df.item_name.str.strip() # removing leading / trailing whitespaces, essential for later steps

items_df['name_1'], items_df['name_2'] = items_df['item_name'].str.split('[', 1).str
items_df['name_1'], items_df['name_3'] = items_df['item_name'].str.split('(', 1).str

# Replace non-alphabatic symbols by ' '
items_df['name_2'] = items_df['name_2'].str.replace('[^A-Za-z0-9А-Яа-я]+', ' ').str.lower().str.strip()
items_df['name_3'] = items_df['name_3'].str.replace('[^A-Za-z0-9А-Яа-я]+', ' ').str.lower().str.strip()
items_df.fillna('0', inplace=True)
items_df.loc[items_df.name_2 == '', 'name_2'] = '0' # two of those '' can not be filled by '0', weird

# Experiments ?
# cnt = collections.Counter(' '.join(items_df['name_2'].values.tolist()).split(' ')).items()
# cnt = sorted(cnt, key=operator.itemgetter(1))
# items_df.sample(10)
# cnt

items_df['type'] = items_df.name_2.apply(lambda x: x[0:8] if x.split(' ')[0] == 'xbox' else x.split(' ')[0])
items_df.loc[(items_df.type == 'x360') | (items_df.type == 'xbox360'), 'type'] = 'xbox 360'
items_df.loc[items_df.type == 'рs3', 'type'] = 'ps3' # beware, 'рs3' is not English string 'ps3'
items_df.loc[(items_df.type == 'pс') | (items_df.type == 'рс'), 'type'] = 'pc' 


items_df[items_df.name_2 == '']
# items_df[(items_df.name_2.str.contains('pc цифровая версия'))].sample(10)
droplists = set([k for k, v in items_df.type.value_counts().items() if v < 20])
droplists
# 
# Replece low frequence item type by other in name_2
items_df['name_2'] = items.type.apply(lambda n: 'other' if n in droplists else n)
items_df['name_2'] = sklearn.preprocessing.LabelEncoder().fit_transform(items_df.name_2)

# name_3 no reducing ??
items_df['name_3'] = sklearn.preprocessing.LabelEncoder().fit_transform(items_df.name_3)
# items_df.name_3.value_counts()

items_df = items_df[['item_id', 'item_category_id', 'name_2', 'name_3']]
items_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  


,item_id,item_category_id,name_2,name_3
0,0,40,0,1318
1,1,76,3,0
2,2,40,0,985
3,3,40,0,984
4,4,40,0,1563
...,...,...,...,...
22165,22165,31,3,0
22166,22166,54,10,0
22167,22167,49,0,575
22168,22168,62,0,0


Dealing with month sales 

In [202]:
import itertools 
products = []
cols = ['date_block_num','shop_id','item_id']

for i in range(34):
    sales = train[train.date_block_num == i]
    products.append(np.array(list(itertools.product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype='int16'))

products_df = pd.DataFrame(np.vstack(products), columns=cols)
products_df.sort_values(cols, inplace=True)
products_df

,date_block_num,shop_id,item_id
114838,0,2,19
117076,0,2,27
120551,0,2,28
118242,0,2,29
114531,0,2,32
...,...,...,...
10667853,33,59,22162
10668043,33,59,22163
10668709,33,59,22164
10670235,33,59,22166


Extract more features on train

In [199]:
train_df = train
train_df['revenue'] = train_df['item_price'] * train['item_cnt_day']
group = train_df.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_cnt_day': ['sum']})

train_df
# TODO 
# https://www.kaggle.com/meliao/lgb-baseline-top8-clear-code-50-faster

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue
0,02.01.2013,0,59,22154,999.00,1.0,999.00
1,03.01.2013,0,25,2552,899.00,1.0,899.00
3,06.01.2013,0,25,2554,1709.05,1.0,1709.05
4,15.01.2013,0,25,2555,1099.00,1.0,1099.00
5,10.01.2013,0,25,2564,349.00,1.0,349.00
...,...,...,...,...,...,...,...
2935838,10.10.2015,33,25,7409,299.00,1.0,299.00
2935839,09.10.2015,33,25,7460,299.00,1.0,299.00
2935840,14.10.2015,33,25,7459,349.00,1.0,349.00
2935841,22.10.2015,33,25,7440,299.00,1.0,299.00


In [200]:
group.columns = ['item_cnt_month']
group.reset_index(inplace=True)
group
# list(itertools.product([2], [4,5], [9,8,7]))


,date_block_num,shop_id,item_id,item_cnt_month
0,0,2,27,1.0
1,0,2,33,1.0
2,0,2,317,1.0
3,0,2,438,1.0
4,0,2,471,2.0
...,...,...,...,...
1607368,33,59,22087,6.0
1607369,33,59,22088,2.0
1607370,33,59,22091,1.0
1607371,33,59,22100,1.0
